In [41]:
import argparse
import pickle
from pathlib import Path

import fraud_eagle as feagle
import fraudar
import networkx as nx
import numpy as np
import pandas as pd
import rsd

from rev2 import rev2compute
from utils import load_data, split_data_by_time

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn.metrics

In [42]:
def get_results(data_name="alpha", alg="fraudar", attack_name="ilp_attack", budget=100):
    data_nw_df, data_gt_df = load_data(data_name=data_name)
    p = Path(f"../res/{attack_name}/{alg}-{data_name}/{budget}.pkl")
    if not p.exists():
        print(p)
        return None
    with open(p, "rb") as fp:
        scores = pickle.load(fp)
        scores = dict(zip(range(len(scores)), scores))
        # 1 means fraudster, 0 means benign
        scores["truth"] = dict(zip(data_gt_df["id"], (1-data_gt_df["label"])/2))
    # df = pd.DataFrame({sp: {u: scores[sp][u] for u in scores[sp] if u in data_gt_df["id"].tolist()} for sp in scores})
    df = pd.DataFrame({sp: {u: scores[sp][u] for u in scores[sp]} for sp in scores})
    df.loc[[u[:5]=="usock" for u in df.index.tolist()], "truth"] = 1
    df.loc[[u[:6]=="udummy" for u in df.index.tolist()], "truth"] = 1
    return df

In [54]:
def get_metrics(df_res, splits=4, K=None, th=0.5):
    if df_res is None:
        return None
    ret = {}
    for sp in range(splits):
        # df_res["truth"] = df_res["truth"].fillna(1)
        df_res.loc[[u[:5]=="usock" for u in df_res.index.tolist()], "truth"] = 1
        df_res.loc[[u[:6]=="udummy" for u in df_res.index.tolist()], "truth"] = 1
        df = df_res[[sp, "truth"]].dropna()
        t = df["truth"].sum() if K is None else K
        t_frac = t / df.shape[0]
        # df["pred"] = (df[sp].rank(ascending=False, method="min") <= t) * 1
        # df["pred"] = df[sp] >= t_frac
        df["pred"] = df[sp] >= th

        ret[sp] = {
            "precision": sklearn.metrics.precision_score(y_true=df["truth"], y_pred=df["pred"]),
            "recall": sklearn.metrics.recall_score(y_true=df["truth"], y_pred=df["pred"]),
            "f1": sklearn.metrics.f1_score(y_true=df["truth"], y_pred=df["pred"])
        }
    return ret

In [55]:
alg_list = ["fraudar", "rsd", "rev2"][:3]
data_list = ["alpha", "otc", "amazon", "epinions"][:2]
budget_list = [100.0, 200.0, 300.0, 400.0]
frac_list = [0.0, 0.2, 0.4, 0.6, 0.8]

In [56]:
attack_res = {
    (d, a, b): get_results(data_name=d, alg=a, attack_name="sockfarm_attack", budget=b)
    for d in data_list for a in alg_list
    for b in budget_list
}

../res/sockfarm_attack/fraudar-alpha/100.0.pkl
../res/sockfarm_attack/fraudar-alpha/200.0.pkl
../res/sockfarm_attack/fraudar-alpha/300.0.pkl
../res/sockfarm_attack/fraudar-alpha/400.0.pkl
../res/sockfarm_attack/rsd-alpha/300.0.pkl
../res/sockfarm_attack/rsd-alpha/400.0.pkl
../res/sockfarm_attack/fraudar-otc/100.0.pkl
../res/sockfarm_attack/fraudar-otc/200.0.pkl
../res/sockfarm_attack/fraudar-otc/300.0.pkl
../res/sockfarm_attack/fraudar-otc/400.0.pkl
../res/sockfarm_attack/rsd-otc/100.0.pkl
../res/sockfarm_attack/rsd-otc/200.0.pkl
../res/sockfarm_attack/rsd-otc/300.0.pkl
../res/sockfarm_attack/rsd-otc/400.0.pkl


In [46]:
th_dict = {
    "fraudar": 0.5,
    "rsd": 1.2,
    "rev2": 0.1,
}
dict_met = {
    (d, a, b): pd.DataFrame.from_dict(get_metrics(df_res=attack_res[(d, a, b)], splits=4, K=None, th=th_dict[a]))
    for d, a, b in attack_res
#     for b in budget_list for f in frac_list
}
df_met = pd.concat(dict_met, keys=dict_met.keys())
display(df_met)

0         1         2         3
alpha rev2 100.0 precision  0.547368  0.556338  0.385027  0.366667
                 recall     1.000000  0.981366  0.464516  0.442953
                 f1         0.707483  0.710112  0.421053  0.401216
           200.0 precision  0.646575  0.653846  0.385027  0.366667
                 recall     1.000000  0.987552  0.306383  0.288210
                 f1         0.785358  0.786777  0.341232  0.322738
           300.0 precision  0.710112  0.382353  0.385027  0.366667
                 recall     1.000000  0.242991  0.228571  0.213592
                 f1         0.830486  0.297143  0.286853  0.269939
           400.0 precision  0.754286  0.382353  0.387097  0.366667
                 recall     1.000000  0.194514  0.182278  0.169666
                 f1         0.859935  0.257851  0.247849  0.231986
otc   rev2 100.0 precision  0.469565  0.483871  0.521277  0.513812
                 recall     0.553846  0.546875  0.529730  0.516667
                 f1         0.508235  0.513447  0.525469  0.515235
           200.0 precision  0.469565  0.483871  0.521277  0.513812
                 recall     0.392727  0.386029  0.369811  0.357692
                 f1         0.427723  0.429448  0.432671  0.421769
           300.0 precision  0.469565  0.483871  0.521277  0.513812
                 recall     0.304225  0.298295  0.284058  0.273529
                 f1         0.369231  0.369069  0.367730  0.357006
           400.0 precision  0.469565  0.483871  0.521277  0.513812
                 recall     0.248276  0.243056  0.230588  0.221429
                 f1         0.324812  0.323575  0.319739  0.309484

In [49]:
def get_alg_data_met(df_met, data="alpha", met="f1"):
    df_met.index = df_met.index.set_names(["data", "algo", "budget", "metrics"])
    rdf = df_met.mean(axis=1).to_frame().reset_index().pivot(index=["data", "algo", "metrics"], columns=["budget"]).loc[(data, slice(None), met)].droplevel(0, axis=1).droplevel([0, 2], axis=0)
    rdf.index = [f"sockfarm-{i}" for i in rdf.index]
    return rdf

In [51]:
get_alg_data_met(df_met, data="alpha", met="f1")

budget,100.0,200.0,300.0,400.0
sockfarm-rev2,0.559966,0.559026,0.421105,0.399405
